<a href="https://colab.research.google.com/github/seyeonjungGit/Wello_team1/blob/main/wee2_material/Week2_2_torch_tutorial_2_sy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import sys
import pandas as pd
import numpy as np
import torch

In [4]:
import torch.nn as nn
import torch.nn.functional as F

In [5]:
# 토치 뉴럴 네트워크 = nn

# Pytorch Graph
`torch.nn` 모듈은 텐서 그래프를 생성하는 다양한 함수를 제공한다. 

[OFFICAL DOCUMENT](https://pytorch.org/docs/stable/nn.html)

# Table of Contents
1. [Container](#Container)
2. [Layers](#Layers)
3. [Loss](#Loss)
4. [추가](#To-Learn-More..)




## Container
- [OFFICIAL DOC](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#module)

layer들을 어떻게 쌓아서 하나의 model Container로 만들 것인가.

Container -> 뼈대. 개별적인 layer들을 어떻게 lapping해서 하나의 모델 컨테이너로 만들 것인가. 

### `Module` class 
- Custom Neural Network를 생성할 때 반드시 `Module` 클래스를 부모 클래스로 상속받아야 함(nn.Module)
- 왜냐하면 `Module` 부모 클래스의 변수 및 메소드 사용 가능 (ex. `eval()`, `train()`, `parameters()`, `state_dict()`, `to()`)
- `forward()` 메소드는 모든 자식클래스에서 반드시 **오버라이딩** 해야함
- [출처](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)

In [6]:
# train() -> True-> require gradiant 가 계속 저장됨 : 백워드시 파라미터가 계속 저장됨.
# parameters() -> 학습되는 파라미터와 파라미터의 weight
# to -> cpu? gpu? 뭐에 올릴 건지.
# foward 메서드는 자식클래스에서 반드시 오버라이딩 해야한다.

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_model.jpeg?raw=true" alt="model" width=600>


In [24]:
class Model(nn.Module): 
    def __init__(self, input_shape):  #input_shape = input의 차원수
        super(Model, self).__init__()  # initialize
        self.layer1 = nn.Linear(input_shape, 32)  # output 32
        self.layer2 = nn.Linear(32, 64)  # output 64
        self.layer_out = nn.Linear(64,1)
        
        self.relu = nn.ReLU()  # activation

    def forward(self, x):  # x= input  # forward 오버라이딩을 하면서 input, output값 지정
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer_out(x)
        return x

In [25]:
model = Model(30) 

for param in model.parameters():
    print(param.shape)

for name, state in model.state_dict().items():  # 학습되는 파라미터 이름과, weight를 출력해준다. 
    print(f"{name} -> size : {state.shape}")

# 학습안된 상태에서는 초기화값만.
# parameters() : https://easy-going-programming.tistory.com/11
# state_dict() : https://tutorials.pytorch.kr/recipes/recipes/what_is_state_dict.html

torch.Size([32, 30])
torch.Size([32])
torch.Size([64, 32])
torch.Size([64])
torch.Size([1, 64])
torch.Size([1])
layer1.weight -> size : torch.Size([32, 30])
layer1.bias -> size : torch.Size([32])
layer2.weight -> size : torch.Size([64, 32])
layer2.bias -> size : torch.Size([64])
layer_out.weight -> size : torch.Size([1, 64])
layer_out.bias -> size : torch.Size([1])


In [9]:
# 내 컴퓨터 환경에서 cuda 쓸 수 있어?
torch.cuda.is_available()  

# cuda 쓰려면 내 컴퓨터에 gpu가 Nbdia?? , cuda, cudn이 설치되어 있dj야 한다. 

True

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model.to(device)  # 커스터마이징한 모델에 device에 올라가게 한다. 이 device에서 연산되는 것.

cuda


Model(
  (layer1): Linear(in_features=30, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

지금까지는 nn.Module 클래스를 상속받아서 커스터마이징한 뉴럴 네트워크를 만드는 방법. 레이어를 변경해서 커스터마이징 모델 만들때는 상속받아야 한다. 

객체지향. 정의한 다음 input, output 어떻게 설정할지 forward오버라이딩 하면서 설정함. 아래의 시퀀셜 클래스는 어떨까? 

### `Sequntial` class
- 여러 layer를 체인처럼 연결한 container
- 이전 layer의 output이 다음 layer의 input으로 입력됨 (순차적)
- [출처](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html#torch.nn.Sequential)

넣은 순서대로 연결된다. 
output이 input이 된다.
한번에 연산되므로 간편하다는 장점이 있음. 

In [11]:
model = nn.Sequential(  # 
          nn.Linear(30, 32),  # 입력 30, output 32
          nn.ReLU(),
          nn.Linear(32,64),  # 입력 32, output 64
          nn.ReLU(),
          nn.Linear(64,1)  # 입력 64, output 1
        )

In [12]:
model.eval()

Sequential(
  (0): Linear(in_features=30, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=1, bias=True)
)

In [13]:
# 인풋값 넣기 x = tensor
model(x)

NameError: ignored

## Layers
- Linear()
    - `input @ weight.T + bias`
https://wikidocs.net/55409


- LSTM()
    - [OFFICAL DOCS](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) 
    - nn.LSTM(`input_size`, `hidden_size`)
        - nn.LSTM()(`input`, (`h_0`, `c_0`))
        - `input` shape: (문장 길이, 배치 사이즈, 단어 엠베딩 사이즈 == input_size) 항상 3차원 (xt)
        - `hidden_size` shape: (lstm 개수 * 레이어 수 , 배치 사이즈, 히든 사이즈 == hidden size) -> 히든state와 cell state의 shape을 튜플로 묶어서 정의해달라.
    - <img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_lstm.png?raw=true" width=500>
    - [출처](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
    

In [14]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

In [15]:
model = nn.Linear(20,30) # (★shape20 을 받아서 shape30을 출력한다. ) (input_dim, output_dim) 20을 받아서 30을 출력  # input(1,20) @ (20,30)  -> output (1,30) + bias(30) (브로드캐스팅)
print(f"W shape: {model.weight.shape}")
print(f"bias shape: {model.bias.shape}")

W shape: torch.Size([30, 20])
bias shape: torch.Size([30])


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [17]:
ex = "I love coding . Just kidding."
inputs = ex.split()
print(inputs)

# 문장길이, 배치사이즈, 단어임베딩사이즈 : (7, 1, 5) 

['I', 'love', 'coding', '.', 'Just', 'kidding.']


In [18]:
input_embedding = [torch.randn(1, 5) for _ in range(len(inputs))]
input_embedding

[tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]]),
 tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]]),
 tensor([[-0.8923, -0.0583, -0.1955, -0.9656,  0.4224]]),
 tensor([[ 0.2673, -0.4212, -0.5107, -1.5727, -0.1232]]),
 tensor([[ 3.5870, -1.8313,  1.5987, -1.2770,  0.3255]]),
 tensor([[-0.4791,  1.3790,  2.5286,  0.4107, -0.9880]])]

In [28]:
input_embedding = [torch.randn(1, 5) for _ in range(len(inputs))]  # 임의의 벡터를 만든다. 

lstm = nn.LSTM(5,5) # (input dim, output dim)  -> input imbedding, hidden states의 dimension을 의미함.
hidden = (
    torch.randn(1,1,5), # (모든 레이어의 lstm 개수(여기서는 단방향이므로 1개), batch size, hidden_size)   # 임베딩 사이즈처럼 알아서 정하기. hidden_size
    torch.randn(1,1,5),  # 만약 역방향으로 쌓는 lstm이 있다면? 순차, 역차를 concat : bilstm. // ElMo방식 -> layer 2개가 된다. 
)

# 한 단어씩 입력 # 토큰 하나씩 lstm 셀에 넣다.
for idx, i in enumerate(input_embedding):   # 한 단어씩 들어간다. 
    out, hidden = lstm(i.view(1,1,-1), hidden)  # Xt 단어('I')가 3차원에 맞춰서 들어가고, hidden states에 initialize한 hidden이 들어간다. -> 그 다음 토큰('love')이 들어갈 때 hidden은 'I'의 히든이 들어간다. 
    print(f"{idx+1} word : output shape ({out.shape}) / hidden state shape ({hidden[0].shape})")

assert out.detach().equal(hidden[0].detach())    # 마지막 layer의 hidden과 output은 동일한 텐서로 나오게 된다. 

print("----------------------------------")

# sequence를 입력 # 문장 통째로.
input_embedding = torch.cat(input_embedding).view(len(inputs), 1, -1)  # torch.cat(input_embedding) // .view(len(inputs), 1, -1)
print(f"input sequence shape : {input_embedding.shape}")
hidden = (
    torch.randn(1,1,5), # (모든 레이어의 lstm 개수, batch size, hidden_size)
    torch.randn(1,1,5),
)
out, hidden = lstm(input_embedding, hidden)  # out = lstm셀 하나에 output으로 나오는 값. hidden = (ht, Ct) 튜플로 엮여서 나온다. 
print(f"output shape : {out.shape}")
print(f"hidden shape : {hidden[0].shape}")  

assert out[-1, : , :].detach().equal(hidden[0][-1, :,:].detach())     # 마지막 워드의 인덱싱 == 히든 마지막은 같다.

# 이해안되면 : https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM  이거 추천(신승연)

1 word : output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
2 word : output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
3 word : output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
4 word : output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
5 word : output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
6 word : output shape (torch.Size([1, 1, 5])) / hidden state shape (torch.Size([1, 1, 5]))
----------------------------------
input sequence shape : torch.Size([6, 1, 5])
output shape : torch.Size([6, 1, 5])
hidden shape : torch.Size([1, 1, 5])


In [36]:
input_embedding = [torch.randn(1, 5) for _ in range(len(inputs))]  # 임의의 벡터를 만든다. 

In [41]:
torch.cat(input_embedding)

tensor([[ 0.5695, -0.1723, -1.9480,  1.2973,  0.7256],
        [ 0.0025, -1.5262, -0.1852, -1.5792, -0.4129],
        [ 0.2975, -0.8007,  2.1303,  0.9013, -0.2871],
        [ 1.1855,  0.1707, -0.3378, -0.5954,  0.0901],
        [-0.8346, -1.7004, -0.4168,  0.2970,  1.1335],
        [ 0.6245,  0.4144, -2.1325,  0.2176, -1.0425]])

In [37]:
torch.cat(input_embedding).view(len(inputs), 1, -1)

tensor([[[ 0.5695, -0.1723, -1.9480,  1.2973,  0.7256]],

        [[ 0.0025, -1.5262, -0.1852, -1.5792, -0.4129]],

        [[ 0.2975, -0.8007,  2.1303,  0.9013, -0.2871]],

        [[ 1.1855,  0.1707, -0.3378, -0.5954,  0.0901]],

        [[-0.8346, -1.7004, -0.4168,  0.2970,  1.1335]],

        [[ 0.6245,  0.4144, -2.1325,  0.2176, -1.0425]]])

In [42]:
torch.cat(input_embedding).view(len(inputs), 1, -1)

tensor([[[ 0.5695, -0.1723, -1.9480,  1.2973,  0.7256]],

        [[ 0.0025, -1.5262, -0.1852, -1.5792, -0.4129]],

        [[ 0.2975, -0.8007,  2.1303,  0.9013, -0.2871]],

        [[ 1.1855,  0.1707, -0.3378, -0.5954,  0.0901]],

        [[-0.8346, -1.7004, -0.4168,  0.2970,  1.1335]],

        [[ 0.6245,  0.4144, -2.1325,  0.2176, -1.0425]]])

In [29]:
hidden

(tensor([[[ 0.1384,  0.2476,  0.1431,  0.3465, -0.0565]]],
        grad_fn=<StackBackward0>),
 tensor([[[ 0.1914,  0.3582,  0.2305,  0.6437, -0.2519]]],
        grad_fn=<StackBackward0>))

In [30]:
out

tensor([[[ 0.0710,  0.3740,  0.4165, -0.1601, -0.0503]],

        [[ 0.1865,  0.2676, -0.0076,  0.2607, -0.1015]],

        [[-0.0075,  0.1121, -0.0720,  0.0740, -0.1688]],

        [[ 0.1752, -0.0274, -0.2696,  0.3723, -0.0740]],

        [[-0.0888, -0.0239, -0.2361,  0.1497, -0.1245]],

        [[ 0.1384,  0.2476,  0.1431,  0.3465, -0.0565]]],
       grad_fn=<StackBackward0>)

## Activation
- nonlinear activations

In [20]:
nn.LeakyReLU()
nn.ReLU()
nn.Sigmoid()
nn.GELU()
nn.Tanh()
nn.Softmax()

Softmax(dim=None)

## Loss
- loss = loss_class()
    - loss(`y_hat`, `y`)
    - `loss().backward()`
- Mean Square Error Loss 
    - <img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_mse.png?raw=true" width=200>
- Cross Entropy Loss 
    - <img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_ce.png?raw=true" width=200>
- Binary Cross Entropy Loss 
    - <img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week2/week2_2_bce.png?raw=true" width=500>

In [ ]:
# 평균제곱오차 : output과 실제 예측값과의 차이의 제곱의 평균.

# corssentropy : 분류에 많이 쓰이는 loss
# t = 실제값
# f(s)i = 예측값
# C = 클래스 개수
# loss 쓰이는 이유 ? : loss함수를 미분하기 때문. 뉴럴 네트워크 만들 떄 학습하는 파라미터(weight) 업뎃 기준이 바로 loss.loss
# 어떤 loss를 쓰느냐에 따라 내 모델이 어떤 방식으로 학습하느냐가 정해진다. 



In [43]:
# l2 distance loss
nn.MSELoss()
nn.NLLLoss()

# cross entropy
## multi class
nn.CrossEntropyLoss() # softmax O -> 자동으로 해결된다. output레이어를 굳이 softmax()해주지 않아도 자동실행.

## binary class
nn.BCELoss()  # -> 활성화함수에output을 넣어야한다.
nn.BCEWithLogitsLoss() # sigmoid O + BCELoss() -> sigmoid 자동해결

BCEWithLogitsLoss()

In [22]:
batch_size = 3
C = 5
logits = torch.randn(batch_size, C, requires_grad=True)  #데이터3개, 클래스 5개 (3,5)shape., requires_grad=True : grad을 이용할거고, 역전파때 이 수치들을 뽑아놔야 한다. -> backward() 메서드 한번만으로 logit의 학습 파라미터가 자동으로 계산된다. 
print(f"Logits : {logits}")   # 정답의 확률값.

loss = nn.CrossEntropyLoss()  

target = torch.empty(batch_size, dtype = torch.long).random_(C)
print(f"Target: {target}") # 인덱스2의 레이블이 정답이다. 인덱스 2가 정답이다. 인덱스 3이 정답이다. -> 타겟을 가리킴.

loss = loss(logits, target)
print(f"Loss : {loss}")  # 확률과 정답을 비교하면서, 가중치 수정
 
loss.backward()

Logits : tensor([[ 0.2214, -0.0558,  1.2057,  1.9486, -0.0766],
        [-0.8562, -0.7870, -0.8161,  0.5470, -1.1707],
        [-0.4699, -1.6271, -0.1127,  1.5980, -0.8445]], requires_grad=True)
Target: tensor([2, 2, 3])
Loss : 1.2612911462783813


## To Learn More...

In [44]:
# dropout
nn.Dropout()
# normalization
nn.BatchNorm1d()

TypeError: ignored

In [ ]:
# gradient clipping  : 업데이트를 배치마다 할건지, 조금더 기다렸다 할건지?
nn.utils.clip_grad()
# weight normalizing
nn.utils.weight_norm()

### Parallel
- 여러 gpu, 또는 여러 머신에서 입력 데이터를 분산 처리를 가능하게 함

현재는 하나의 머신에서 cpu 연산

서버에서는 여러 머신에 대해 한 머신이 여러 gpu를 가지고 있다. 그 gpu를 어떻게 병렬적으로 사용할까? 

여러 머신 여러 gpu의 gradient를 모아서 계산해야 한다. 

In [ ]:
nn.DataParallel()    
nn.parallel.DistributedDataParallel()